In [1]:
import shioaji as sj
from pymongo import MongoClient
from datetime import datetime, timedelta
from pandas import DataFrame
import numpy as np
from Messenger.LineMessenger import LineMessenger as Line
from time import sleep

import warnings
warnings.filterwarnings("ignore")

from prettytable import PrettyTable
import json
import os 
parent = os.path.dirname(os.path.abspath("__file__"))
StrongPath = os.path.join(parent, 'StrongTickers')
if not os.path.isdir(StrongPath):
    os.makedirs(StrongPath)

setting = {
    'user':'kevin83321',
    'pwd':'j7629864',
    'ip':'192.168.2.173',
    'port':'49153'
}

In [2]:
def get_minimum_tick(cost):
    if cost < 10:
        return 0.01
    elif cost < 50:
        return 0.05
    elif cost < 100:
        return 0.1
    elif cost < 500:
        return 0.5
    elif cost < 1000:
        return 1
    else:
        return 5
    
def get_commission(price:float, multiplier:int=1000, qty=1, Real:bool=True, direction:str='', dayTrade:bool=False):
    """
    計算個別部位的單邊交易成本

    Params:
        symbol : 商品代碼
        exchange : 交易所
        cost : 交易價格
        multiplier : 價格還原現金之乘數
            例如:
                股票 : 1張 = 1,000股，10元的股票還原現金價值，即為10 *1,000 = 10,000元
                期貨 : 台指期1點200元，假設現在10,000點，則一口台股的價值為 200 * 10,000 = 2,000,000
        qty : 買賣口數或張數
        Real : 是否為實單, default = False
        direction : 交易方向 進場(買賣)或出場
            P.S. 股票交易的交易稅是出場才計算
    """
    tick = get_minimum_tick(price)
    commission = price * (0.1425 / 100) * multiplier * qty
    commission = 20 if commission < 20 else commission
    fee = price * (0.3 / 100) * multiplier * qty
    if dayTrade:
        fee /= 2
    slide = tick * multiplier
    tradeCost = commission * 0.6
    if direction == 'EXIT' or direction == 0:
        tradeCost += fee
    if not Real:
        tradeCost += slide * qty
    return tradeCost

In [3]:
api = sj.Shioaji()

In [4]:
api.login("F128497445", "89118217k")

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


[FutureAccount(person_id='F128497445', broker_id='F002000', account_id='1473661', signed=True, username='鄭圳宏'),
 Account(account_type=<AccountType.H: 'H'>, person_id='F128497445', broker_id='9A92', account_id='0011645', username='鄭圳宏\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000'),
 StockAccount(person_id='F128497445', broker_id='9A92', account_id='0231901', signed=True, username='鄭圳宏\u3000\u3000')]

In [5]:
ODD_Data_BA = {}
Tick_Data_BA = {}
spread = {}

In [6]:
from shioaji import TickSTKv1, Exchange, BidAskSTKv1, TickFOPv1, BidAskFOPv1

# @api1.on_tick_stk_v1()
# @api1.on_bidask_stk_v1()
# @api1.on_bidask_fop_v1()
# @api1.on_tick_fop_v1()


def quote_callback(exchange:Exchange, tick:[TickSTKv1, BidAskSTKv1, TickFOPv1, BidAskFOPv1]):
    try:
        if type(tick) in [BidAskSTKv1, BidAskFOPv1] or 1:
            data = dict(
                symbol = tick.code,
                datetime = tick.datetime,
                bid1 = float(tick.bid_price[0]),
                bid2 = float(tick.bid_price[1]),
                bid3 = float(tick.bid_price[2]),
                bid4 = float(tick.bid_price[3]),
                bid5 = float(tick.bid_price[4]),
                bidQty1 = float(tick.bid_volume[0]),
                bidQty2 = float(tick.bid_volume[1]),
                bidQty3 = float(tick.bid_volume[2]),
                bidQty4 = float(tick.bid_volume[3]),
                bidQty5 = float(tick.bid_volume[4]),
                askQty1 = float(tick.ask_volume[0]),
                askQty2 = float(tick.ask_volume[1]),
                askQty3 = float(tick.ask_volume[2]),
                askQty4 = float(tick.ask_volume[3]),
                askQty5 = float(tick.ask_volume[4]),
                ask1 = float(tick.ask_price[0]),
                ask2 = float(tick.ask_price[1]),
                ask3 = float(tick.ask_price[2]),
                ask4 = float(tick.ask_price[3]),
                ask5 = float(tick.ask_price[4]),
                simulate=bool(tick.simtrade),
                odd=bool(tick.intraday_odd)
            )
#             print(data)
            if data['odd']:
                ODD_Data_BA[tick.code] = ODD_Data_BA.get(tick.code, []) + [data]
            else:
                Tick_Data_BA[tick.code] = Tick_Data_BA.get(tick.code, []) + [data]
            if tick.code in ODD_Data_BA.keys() and tick.code in Tick_Data_BA.keys():
                NaOb_r = ODD_Data_BA[tick.code][-1]['bid1'] / Tick_Data_BA[tick.code][-1]['ask1'] - 1 if Tick_Data_BA[tick.code][-1]['ask1'] else 0
                NbOa_r = Tick_Data_BA[tick.code][-1]['bid1'] / ODD_Data_BA[tick.code][-1]['ask1'] - 1 if ODD_Data_BA[tick.code][-1]['ask1'] else 0
                NaOb_s = ODD_Data_BA[tick.code][-1]['bid1'] - Tick_Data_BA[tick.code][-1]['ask1'] if Tick_Data_BA[tick.code][-1]['ask1'] else 0
                NbOa_s = Tick_Data_BA[tick.code][-1]['bid1'] - ODD_Data_BA[tick.code][-1]['ask1'] if ODD_Data_BA[tick.code][-1]['ask1'] else 0
                spd_data = dict(
                    symbol = tick.code,
                    datetime = tick.datetime,
                    NaOb_r=NaOb_r, # 價差比率，賣整買零
                    NbOa_r=NbOa_r, # 價差比率，賣零買整
                    NaOb_s=NaOb_s, # 價差，賣整買零
                    NbOa_s=NbOa_s, # 價差，賣零買整
                    bid1=Tick_Data_BA[tick.code][-1]['bid1'],
                    ask1=Tick_Data_BA[tick.code][-1]['ask1'],
                    odd_bid1=ODD_Data_BA[tick.code][-1]['bid1'],
                    odd_ask1=ODD_Data_BA[tick.code][-1]['ask1'],
                    bidQty1=Tick_Data_BA[tick.code][-1]['bidQty1'],
                    askQty1=Tick_Data_BA[tick.code][-1]['askQty1'],
                    odd_bidQty1=ODD_Data_BA[tick.code][-1]['bidQty1'],
                    odd_askQty1=ODD_Data_BA[tick.code][-1]['askQty1'],
                    simulate=data['simulate'],
                    triggerbyOdd=data['odd'],
                )
                # print(spd_data, '\n')
                spread[tick.code] = spread.get(tick.code, []) + [spd_data]
#             print(f"Exchange : {exchange}, BidAsk : {NotifyTickers[ticker].q80_data}\n")
    except Exception as e:
        print(e)
    
api.quote.set_on_tick_stk_v1_callback(quote_callback)
api.quote.set_on_bidask_stk_v1_callback(quote_callback)
# api.quote.set_on_bidask_fop_v1_callback(quote_callback)
# api.quote.set_on_tick_fop_v1_callback(quote_callback)

def place_cb(stat, msg):
#     NotifyTickers[msg['code']].updateOrderDeal(stat, msg)
    print(f"Order Status : {stat}, Order Data : {msg}")
    
api.set_order_callback(place_cb)

In [7]:
# NotifyTickers = {}
# for i in range(125):
#     if i < len(open_thresholds.keys()):
#         ticker = sorted(open_thresholds.keys())[i]
for ticker in '2330,2317,2603,2615,2609,2303,0056,0050,3481,1101,2884,006208,2887,2886,2890,2409,2891,2883,2812,5880,2892,2610,2618,2882'.split(','):
    tmp_contract = api.Contracts.Stocks[ticker]
    if tmp_contract.day_trade != sj.constant.DayTrade.No:
        print("Tradable Ticker : " + ticker)
        # tmp_contract = api1.Contracts.Futures[ticker]
#             NotifyTickers[ticker] = NotifyTickers.get(ticker,
#                                                       DataObject(tmp_contract, open_threshold=tmp_contract.reference, 
#                                                                 api=api, max_size = max_size_map[ticker], PreData=open_thresholds[ticker]))
#             api.quote.subscribe(
#                 tmp_contract, 
#                 quote_type = sj.constant.QuoteType.Tick, # or 'tick'
#                 version = sj.constant.QuoteVersion.v1 # or 'v1'
#             )
        api.quote.subscribe(
            tmp_contract, 
            quote_type = sj.constant.QuoteType.BidAsk, # or 'tick'
            version = sj.constant.QuoteVersion.v1 # or 'v1'
        )
        api.quote.subscribe(
            tmp_contract, 
            quote_type = sj.constant.QuoteType.BidAsk, # or 'tick'
            version = sj.constant.QuoteVersion.v1, # or 'v1'
            intraday_odd=True
        )

Tradable Ticker : 2330
Tradable Ticker : 2317
Tradable Ticker : 2603
Tradable Ticker : 2615
Tradable Ticker : 2609
Tradable Ticker : 2303
Tradable Ticker : 0056
Tradable Ticker : 0050
Tradable Ticker : 3481
Tradable Ticker : 1101
Tradable Ticker : 2884
Tradable Ticker : 006208
Tradable Ticker : 2887
Tradable Ticker : 2886
Tradable Ticker : 2890
Tradable Ticker : 2409
Tradable Ticker : 2891
Tradable Ticker : 2883
Tradable Ticker : 2812
Tradable Ticker : 5880
Tradable Ticker : 2892
Tradable Ticker : 2610
Tradable Ticker : 2618
Tradable Ticker : 2882
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/2330 | Event: Subscribe or Unsubscribe ok

In [8]:
while 1:
    try:
        sleep(1)
    except:
        break

In [9]:
NotifyTickers = {}
# for i in range(125):
#     if i < len(open_thresholds.keys()):
#         ticker = sorted(open_thresholds.keys())[i]
for ticker in '2330,2317,2603,2615,2609,2303,0056,0050,3481,1101,2884,006208,2887,2886,2890,2409,2891,2883,2812,5880,2892,2610,2618,2882'.split(','):
    tmp_contract = api.Contracts.Stocks[ticker]
    if tmp_contract.day_trade != sj.constant.DayTrade.No:
        print("Tradable Ticker : " + ticker)
        # tmp_contract = api1.Contracts.Futures[ticker]
#             NotifyTickers[ticker] = NotifyTickers.get(ticker,
#                                                       DataObject(tmp_contract, open_threshold=tmp_contract.reference, 
#                                                                 api=api, max_size = max_size_map[ticker], PreData=open_thresholds[ticker]))
#             api.quote.subscribe(
#                 tmp_contract, 
#                 quote_type = sj.constant.QuoteType.Tick, # or 'tick'
#                 version = sj.constant.QuoteVersion.v1 # or 'v1'
#             )
        api.quote.unsubscribe(
            tmp_contract, 
            quote_type = sj.constant.QuoteType.BidAsk, # or 'tick'
            version = sj.constant.QuoteVersion.v1 # or 'v1'
        )
        api.quote.unsubscribe(
            tmp_contract, 
            quote_type = sj.constant.QuoteType.BidAsk, # or 'tick'
            version = sj.constant.QuoteVersion.v1, # or 'v1'
            intraday_odd=True
        )

Tradable Ticker : 2330
Tradable Ticker : 2317
Tradable Ticker : 2603
Tradable Ticker : 2615
Tradable Ticker : 2609
Tradable Ticker : 2303
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/2330 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/ODD/*/TSE/2330 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/2317 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/ODD/*/TSE/2317 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/2603 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/ODD/*/TSE/2603 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/2615 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/ODD/*/TSE/2615 | Event: Subscribe or Unsubscribe okTradable Ticker : 0056
Tradable Ticker 

In [10]:
td = datetime.today()
# td = datetime(2022,10,21)
output_path = os.path.join(parent, 'Output', "整零套利", td.strftime("%Y%m%d"))
if not os.path.isdir(output_path):
    os.makedirs(output_path)

In [11]:
for ticker, tmp_data in spread.items():
    tmp_df = DataFrame(tmp_data)
    tmp_df.to_csv(os.path.join(output_path, f'{ticker}_spread.csv'), index=False, encoding='utf-8')

In [13]:
spread.keys()

dict_keys(['2884', '0056', '1101', '006208', '0050', '2890', '2883', '2409', '2317', '2892', '2886', '2303', '2330', '2609', '2615', '2891', '2610', '2603', '2887', '5880', '3481', '2618', '2812'])

In [12]:
tmp_df = DataFrame(spread['2882'])

# tmp_df[((tmp_df.NaOb_r >= 0.0025) | (tmp_df.NbOa_r >= 0.0025)) & (tmp_df.simulate != True) & (tmp_df.triggerbyOdd)]
tmp_df[((tmp_df.NaOb_r >= 0.0035) | (tmp_df.NbOa_r >= 0.0035))].head(70)#.max()

,symbol,datetime,NaOb_r,NbOa_r,NaOb_s,NbOa_s,bid1,ask1,odd_bid1,odd_ask1,bidQty1,askQty1,odd_bidQty1,odd_askQty1,simulate,triggerbyOdd
203,2882,2022-10-31 09:38:56.159681,0.003958,-0.006562,0.15,-0.25,37.85,37.90,38.05,38.10,114.0,14.0,2271.0,43159.0,False,False
204,2882,2022-10-31 09:38:56.191745,0.003958,-0.006562,0.15,-0.25,37.85,37.90,38.05,38.10,90.0,14.0,2271.0,43159.0,False,False
205,2882,2022-10-31 09:38:56.257525,0.003958,-0.006562,0.15,-0.25,37.85,37.90,38.05,38.10,90.0,14.0,2271.0,43159.0,False,False
206,2882,2022-10-31 09:38:56.308317,0.003958,-0.006562,0.15,-0.25,37.85,37.90,38.05,38.10,90.0,14.0,2271.0,43159.0,False,False
207,2882,2022-10-31 09:38:56.441318,0.003958,-0.006562,0.15,-0.25,37.85,37.90,38.05,38.10,90.0,14.0,2271.0,43159.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7305,2882,2022-10-31 12:48:22.871293,0.003942,-0.006536,0.15,-0.25,38.00,38.05,38.20,38.25,710.0,79.0,3343.0,41089.0,False,False
7306,2882,2022-10-31 12:48:23.174320,0.003942,-0.006536,0.15,-0.25,38.00,38.05,38.20,38.25,709.0,79.0,3343.0,41089.0,False,False
7307,2882,2022-10-31 12:48:23.611253,0.003942,-0.006536,0.15,-0.25,38.00,38.05,38.20,38.25,709.0,80.0,3343.0,41089.0,False,False
7308,2882,2022-10-31 12:48:24.189251,0.003942,-0.006536,0.15,-0.25,38.00,38.05,38.20,38.25,709.0,84.0,3343.0,41089.0,False,False
